In [1]:
import pulp
import numpy as np
import scipy.stats

# 1. Data

In [119]:
# data
p = 7 #selling price
c = 5 #buying cost
min_demand = 50 
max_demand = 80
N = max_demand-min_demand+1 # scenarios
scenarios = [s for s in range(1,N+1)]
demand = [d for d in range(min_demand,max_demand+1)]
demand_scenario = dict(zip(scenarios,demand))
# uniform
proba = [scipy.stats.randint(min_demand,max_demand+1).pmf(d) for d in demand]
#proba = [1/len(scenarios) for s in scenarios]
# normal
proba = [scipy.stats.norm((min_demand+max_demand)/2, 3).pdf(d) for d in demand]
proba_scenario = dict(zip(scenarios,proba))

In [120]:
sum(proba)

0.9999997899653588

In [121]:
# proba_scenario

# 2. Stochastic Program

In [122]:
# create problem
prob = pulp.LpProblem("newsboy", pulp.LpMaximize)

# define variables
buy = pulp.LpVariable("buy",cat=pulp.LpInteger)
sell = pulp.LpVariable.dicts("sell",scenarios,cat=pulp.LpInteger)

# objective function
#prob += p*1/31*pulp.lpSum(sell)-c*buy
prob += p*pulp.lpSum(proba_scenario[s]*sell[s] for s in scenarios)-c*buy

# constraints: sell = min{buy, demand}
for s in scenarios:
    prob += sell[s] <= buy
    prob += sell[s] <= demand_scenario[s]

In [123]:
prob.solve()

1

In [124]:
pulp.value(buy),pulp.value(prob.objective)

(63.0, 122.88871260673636)

In [136]:
# check that solver's solution matches theoretical solution
scipy.stats.randint.ppf((p-c)/p,min_demand,max_demand) #if uniform, or np.quantile(demand,(p-c)/p)
scipy.stats.norm.ppf((p-c)/p, loc=(min_demand+max_demand)/2, scale=3) #if normal

63.30215353420141

# 3. L-Shaped algorithm

In [ ]:
def mastersolve(scenarios, iter):
    prob = pulp.LpProblem("master", pulp.LpMinimize)
    buy = pulp.LpVariable("buy",cat=pulp.LpInteger)
    sell = pulp.LpVariable.dicts("sell",scenarios,cat=pulp.LpInteger)
    theta = pulp.LpVariable("theta",lowBound=0,cat=pulp.Continuous)
    # obj
    prob += c*buy+theta
    # constraints
    for s in scenarios:
        prob += sell[s] <= buy
        prob += sell[s] <= demand_scenario[s]
    if iter>1:
        prob += theta >= pulp.lpSum(proba_scenario[s]*(p*sell[s]-c*buy) for s in scenarios)
    else:
        prob += theta == 0
    prob.solve()
    return pulp.value(buy)

def subsolve(buy,demand_scenario):
    prob = pulp.LpProblem("sub", pulp.LpMaximize)
    sell = pulp.LpVariable("sell",cat=pulp.LpContinuous, lowBound=0)
    prob += p*sell-c*buy
    prob += sell <= buy, "c1"
    prob += sell <= demand_scenario, "c2"
    prob.solve()
    
    return pulp.value(prob.objective)

def L_shaped():
    iter = 1
    tol=1e-3
    UB = 1000
    LB = -1000
    scenarios = []
    while UB-LB>tol:
        buy = mastersolve(scenarios, iter)
        LB = buy
        for s in scenarios:
            sub_obj = subsolve(buy,demand_scenario[s])
            UB = sub_obj

# 4. Convergence depending on nb of scenarios

# 5. VSS vs EVPP vs EVS